In [6]:
# Importing all the required Libraries 
import unidecode 
import pandas as pd 
import re
import time 
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from autocorrect import Speller 
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords 
from nltk import word_tokenize 
import string 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
import spacy
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
df_questions = pd.read_csv(r"D:\Yash-USA\Northeastern University\Applied Artificial Intelligence\Quarter-4 Fall'22\Fall Term - AB\INT 6940 Experiential Learning Projects for Professionals\Repurpost\Dataset\Questions.csv", encoding="ISO-8859-1")
df_questions.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [8]:
df_questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264216 entries, 0 to 1264215
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Id            1264216 non-null  int64  
 1   OwnerUserId   1249762 non-null  float64
 2   CreationDate  1264216 non-null  object 
 3   ClosedDate    55959 non-null    object 
 4   Score         1264216 non-null  int64  
 5   Title         1264216 non-null  object 
 6   Body          1264216 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 67.5+ MB


In [9]:
# Dropping columns that are not necessary for repurpost project
df_questions.drop(columns=['OwnerUserId', 'CreationDate', 'ClosedDate'], inplace=True)
df_questions.head()

,Id,Score,Title,Body
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [10]:
df_tags = pd.read_csv(r"D:\Yash-USA\Northeastern University\Applied Artificial Intelligence\Quarter-4 Fall'22\Fall Term - AB\INT 6940 Experiential Learning Projects for Professionals\Repurpost\Dataset\Tags.csv", encoding="ISO-8859-1")
df_tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [11]:
df_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750994 entries, 0 to 3750993
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Tag     object
dtypes: int64(1), object(1)
memory usage: 57.2+ MB


In [12]:
# The same id is having more than one tag
# to combine this data with the questions, we need to combine the all the tags belonging to an id
# and then join with questions dataframe to avoid cartesion product of the join

# Convert Tag column datatype to string for grouping
df_tags['Tag'] = df_tags['Tag'].astype(str)

# Combining the tags for a particular Id separated by pipe '|' delimiter
df_tags_grouped = pd.DataFrame(df_tags.groupby('Id')['Tag'].apply(lambda tag: '|'.join(tag))).reset_index()
df_tags_grouped.head()

,Id,Tag
0,80,flex|actionscript-3|air
1,90,svn|tortoisesvn|branch|branching-and-merging
2,120,sql|asp.net|sitemap
3,180,algorithm|language-agnostic|colors|color-space
4,260,c#|.net|scripting|compiler-construction


In [13]:
# Check the rows and columns of df_tags_grouped dataframe
df_tags_grouped.shape

(1264216, 2)

In [14]:
# Check the rows and columns of df_question dataframe
df_questions.shape

(1264216, 4)

In [15]:
# Combine the questions and tags for next steps
df_stackoverflow = df_questions.merge(df_tags_grouped, on='Id')
df_stackoverflow.head()

,Id,Score,Title,Body,Tag
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex|actionscript-3|air
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn|tortoisesvn|branch|branching-and-merging
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql|asp.net|sitemap
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,algorithm|language-agnostic|colors|color-space
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c#|.net|scripting|compiler-construction


In [16]:
del df_questions
del df_tags
del df_tags_grouped

In [17]:
df_stackoverflow['Tags'] = df_stackoverflow['Tag'].apply(lambda x: x.split('|'))
df_stackoverflow.head()

,Id,Score,Title,Body,Tag,Tags
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex|actionscript-3|air,"[flex, actionscript-3, air]"
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn|tortoisesvn|branch|branching-and-merging,"[svn, tortoisesvn, branch, branching-and-merging]"
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql|asp.net|sitemap,"[sql, asp.net, sitemap]"
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,algorithm|language-agnostic|colors|color-space,"[algorithm, language-agnostic, colors, color-s..."
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c#|.net|scripting|compiler-construction,"[c#, .net, scripting, compiler-construction]"


In [18]:
tags_list = [tag for tags in df_stackoverflow[df_stackoverflow['Score'] > 5]['Tags'].values for tag in tags]

In [19]:
tag_counts = nltk.FreqDist(tags_list)
top_100_tags = [tag[0] for tag in tag_counts.most_common(100)]
top_100_tags[0:20]

['c#',
 'java',
 'javascript',
 'android',
 'python',
 'c++',
 'php',
 'jquery',
 '.net',
 'ios',
 'html',
 'css',
 'c',
 'iphone',
 'objective-c',
 'ruby-on-rails',
 'sql',
 'asp.net',
 'mysql',
 'ruby']

tag_counts = nltk.FreqDist(tags_list)
top_1000_tags = [tag[0] for tag in tag_counts.most_common(1000)]
top_1000_tags[0:20]

In [20]:
# Define function to take list of tags as input and create a new list only based
# on tags that are part of the top 1000 tags list
def keep_top_tags(tags):
  tags_list = []
  for tag in tags:
    if tag in top_100_tags:
      tags_list.append(tag)

  if (len(tags_list) > 0):
    return tags_list
  else:
    return None

In [21]:
df_stackoverflow['Tags'] = df_stackoverflow['Tags'].apply(lambda tags: keep_top_tags(tags))

In [22]:
df_stackoverflow.head()

,Id,Score,Title,Body,Tag,Tags
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex|actionscript-3|air,None
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn|tortoisesvn|branch|branching-and-merging,[svn]
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql|asp.net|sitemap,"[sql, asp.net]"
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,algorithm|language-agnostic|colors|color-space,[algorithm]
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c#|.net|scripting|compiler-construction,"[c#, .net]"


In [23]:
df_stackoverflow = df_stackoverflow[df_stackoverflow['Score'] > 5].dropna()

In [24]:
# Drop extra columns.
df_stackoverflow.drop(columns=['Tag'], inplace=True)
df_stackoverflow.head()

,Id,Score,Title,Body,Tags
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,[svn]
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net]"
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,[algorithm]
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net]"
5,330,29,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,"[c++, oop, class]"


In [25]:
df_stackoverflow.shape

(63167, 5)

In [26]:
df_stackoverflow.isna().sum()

Id       0
Score    0
Title    0
Body     0
Tags     0
dtype: int64

### Data Cleaning and pre-processing:

In [27]:
df_stackoverflow.head()

,Id,Score,Title,Body,Tags
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,[svn]
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net]"
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,[algorithm]
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net]"
5,330,29,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,"[c++, oop, class]"


In [28]:
df_stackoverflow.shape

(63167, 5)

In [29]:
#Cleaning HTML tags from the title and body text
def clean_html(html):

    # parse html content
    soup = BeautifulSoup(html, "html.parser")

    for data in soup(['style', 'script', 'code', 'a']):
        # Remove tags
        data.decompose()

    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

In [30]:
# To remove HTML first and apply it directly to the source text column.
df_stackoverflow['Body'] = df_stackoverflow['Body'].apply(lambda x: clean_html(x))

In the below code chunk, the data has been passed through a function which cleans the data by making the text into lower case and by removing the line breaks, punctuations, stop words and numbers. And also, stemming and lemmatization is performed. 

In [31]:
# Load spacy
nlp = spacy.load('en_core_web_sm')

def clean_string(text, stem="None"):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', '', text)

    # Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    useless_words = useless_words + ['hi', 'im']

    text_filtered = [word for word in text if not word in useless_words]

    # Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string

In [32]:
# Next apply the clean_string function to the title text
df_stackoverflow['title_clean'] = df_stackoverflow['Title'].apply(lambda x: clean_string(x, stem='Stem'))

In [33]:
df_stackoverflow.head()

,Id,Score,Title,Body,Tags,title_clean
1,90,144,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...,[svn],good branch merg tutori tortoisesvn
2,120,21,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...,"[sql, asp.net]",aspnet site map
3,180,53,Function for creating color wheels,This is something I've pseudo-solved many time...,[algorithm],function creat color wheel
4,260,49,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,"[c#, .net]",ad script function net applic
5,330,29,Should I use nested classes in this case?,I am working on a collection of classes used f...,"[c++, oop, class]",use nest class case


The body text contains punctuations which should not be removed because this attribute helps the models to understand and makes sense with the tags attribute to run the model efficicnetly. So, the following data clean code is rpeated but without punctuiation removal in the 'Body' column

In [34]:
# Load spacy
nlp = spacy.load('en_core_web_sm')

def clean_string_body(text, stem="None"):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', '', text)

    # Remove puncuation
    #translator = str.maketrans('', '', string.punctuation)
    #text = text.translate(translator)

    # Remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    useless_words = useless_words + ['hi', 'im']

    text_filtered = [word for word in text if not word in useless_words]

    # Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string

In [35]:
# Next apply the clean_string function to the body text
df_stackoverflow['body_clean'] = df_stackoverflow['Body'].apply(lambda x: clean_string_body(x, stem='Stem'))

In [36]:
df_stackoverflow.head()

,Id,Score,Title,Body,Tags,title_clean,body_clean
1,90,144,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...,[svn],good branch merg tutori tortoisesvn,realli good tutori explain apach subversion? b...
2,120,21,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...,"[sql, asp.net]",aspnet site map,anyon got experi creat sql-base asp.net site-m...
3,180,53,Function for creating color wheels,This is something I've pseudo-solved many time...,[algorithm],function creat color wheel,someth i'v pseudo-solv mani time never quit fo...
4,260,49,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,"[c#, .net]",ad script function net applic,"littl game written c#. use databas back-end. ,..."
5,330,29,Should I use nested classes in this case?,I am working on a collection of classes used f...,"[c++, oop, class]",use nest class case,work collect class use video playback recordin...


In [37]:
# Saving the cleaned dataset

In [38]:
df_stackoverflow.to_csv("StackoverFlow_Final",index = False)

In [39]:
import pandas as pd
import os.path
df_stackoverflow.to_csv(os.path.join(r"C:\Users\User\OneDrive\Desktop",'StackoverFlow_Final.csv'))

#### Tokenization

In [40]:
df_stackoverflow.head()

,Id,Score,Title,Body,Tags,title_clean,body_clean
1,90,144,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...,[svn],good branch merg tutori tortoisesvn,realli good tutori explain apach subversion? b...
2,120,21,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...,"[sql, asp.net]",aspnet site map,anyon got experi creat sql-base asp.net site-m...
3,180,53,Function for creating color wheels,This is something I've pseudo-solved many time...,[algorithm],function creat color wheel,someth i'v pseudo-solv mani time never quit fo...
4,260,49,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,"[c#, .net]",ad script function net applic,"littl game written c#. use databas back-end. ,..."
5,330,29,Should I use nested classes in this case?,I am working on a collection of classes used f...,"[c++, oop, class]",use nest class case,work collect class use video playback recordin...


In [41]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_stackoverflow['body_clean'])

In [42]:
# check unique words count
len(tokenizer.word_index)

69412

In [43]:
# check unique words count
vocab_size = len(tokenizer.word_index) + 1
vocab_size

69413

In [44]:
sequences = tokenizer.texts_to_sequences(df_stackoverflow['body_clean'])

In [ ]:
for i in range(20):
    print(df_stackoverflow['body_clean'][i], '\n'), print(sequences[i])

In [46]:
seq_lengths = []

for i in sequences:
    seq_lengths.append(len(i))

In [47]:
print("30th percentile: ", pd.Series(seq_lengths).quantile(0.3))
print("40th percentile: ", pd.Series(seq_lengths).quantile(0.4))
print("50th percentile: ", pd.Series(seq_lengths).quantile(0.5))
print("60th percentile: ", pd.Series(seq_lengths).quantile(0.6))
print("70th percentile: ", pd.Series(seq_lengths).quantile(0.7))
print("80th percentile: ", pd.Series(seq_lengths).quantile(0.8))
print("90th percentile: ", pd.Series(seq_lengths).quantile(0.9))
print("95th percentile: ", pd.Series(seq_lengths).quantile(0.95))
print("99th percentile: ", pd.Series(seq_lengths).quantile(0.99))

30th percentile:  26.0
40th percentile:  32.0
50th percentile:  39.0
60th percentile:  47.0
70th percentile:  58.0
80th percentile:  74.0
90th percentile:  103.0
95th percentile:  136.0
99th percentile:  237.3399999999965


In [48]:
from sklearn.model_selection import GridSearchCV
from keras_preprocessing.sequence import pad_sequences

from keras.wrappers.scikit_learn import KerasClassifier
max_length = 125

# padding
padded_seq = pad_sequences(sequences, maxlen=max_length)

#### MultiLabel Binarization

In [49]:
#separating and storing tags into a variable
tags = df_stackoverflow['Tags']

In [50]:
tags

1                      [svn]
2             [sql, asp.net]
3                [algorithm]
4                 [c#, .net]
5          [c++, oop, class]
                 ...        
1262834          [c++, list]
1262915             [python]
1263065             [python]
1263454                [c++]
1264205            [haskell]
Name: Tags, Length: 63167, dtype: object

In [51]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_stackoverflow['Tags'])
y = multilabel_binarizer.transform(df_stackoverflow['Tags'])

In [52]:
#tags after binarization are stored as an array
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [53]:
multilabel_binarizer.classes_

array(['.net', 'ajax', 'algorithm', 'android', 'angularjs', 'api',
       'arrays', 'asp.net', 'asp.net-mvc', 'asp.net-mvc-3', 'bash', 'c',
       'c#', 'c++', 'c++11', 'class', 'cocoa', 'cocoa-touch', 'css',
       'css3', 'database', 'datetime', 'debugging', 'delphi',
       'design-patterns', 'django', 'eclipse', 'emacs',
       'entity-framework', 'exception', 'facebook', 'function', 'gcc',
       'generics', 'git', 'github', 'google-chrome', 'haskell',
       'hibernate', 'html', 'html5', 'http', 'image', 'ios', 'ipad',
       'iphone', 'java', 'javascript', 'jquery', 'json', 'linq', 'linux',
       'list', 'math', 'maven', 'mongodb', 'multithreading', 'mysql',
       'node.js', 'numpy', 'objective-c', 'oop', 'optimization', 'oracle',
       'osx', 'performance', 'perl', 'php', 'postgresql', 'python', 'qt',
       'r', 'regex', 'rest', 'ruby', 'ruby-on-rails', 'ruby-on-rails-3',
       'scala', 'security', 'shell', 'spring', 'sql', 'sql-server',
       'string', 'svn', 'swift', 's

In [85]:
#binarized tags as a dataframe
y = pd.DataFrame(y, columns=multilabel_binarizer.classes_)
y.shape

(63167, 100)

In [86]:
padded_seq.shape, y.shape

((63167, 125), (63167, 100))

In [87]:
#TFIDF
#from sklearn.feature_extraction.text import TfidfVectorizer


In [88]:
#tf idf
#tf_idf = TfidfVectorizer()
#applying tf idf to body text
#body_tf = tf_idf.fit_transform(df_stackoverflow['body_clean'])

In [58]:
#print(body_tf)

In [59]:
#applying tf idf to title text
#title_tf = tf_idf.fit_transform(df_stackoverflow['title_clean'])

In [60]:
#print(title_tf)

In [61]:
#df_stackoverflow.head()

##### Tf-IDF Vectorization

In [62]:
#TfIDF

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=5000)

vectorizer = TfidfVectorizer()

In [90]:
tfidf = TfidfVectorizer(analyzer='word', max_features=10000, ngram_range=(1,3), stop_words='english')
X = tfidf.fit_transform(df_stackoverflow['body_clean'])

In [91]:
print(X)

  (0, 3498)	0.38235957664001674
  (0, 6698)	0.37912224898036917
  (0, 1281)	0.2181781422361352
  (0, 8465)	0.4004734843229102
  (0, 7757)	0.19695433710765312
  (0, 788)	0.19011577145165773
  (0, 8028)	0.39471563454309383
  (0, 388)	0.2947858881593386
  (0, 2845)	0.21761736485741462
  (0, 8652)	0.2639591108971749
  (0, 3486)	0.18863205112588743
  (0, 6691)	0.17843433298694983
  (1, 5865)	0.22752941681836625
  (1, 9617)	0.224258088618587
  (1, 5441)	0.18650931731620124
  (1, 4772)	0.21409672167629673
  (1, 9829)	0.20226635243037958
  (1, 3085)	0.18744455301359064
  (1, 9957)	0.16480854353307808
  (1, 588)	0.13583026430324222
  (1, 357)	0.2172379817727759
  (1, 5088)	0.21409672167629673
  (1, 7832)	0.12365461399260559
  (1, 5995)	0.16314017328753988
  (1, 9351)	0.10760222608494309
  :	:
  (63166, 7622)	0.09999219855650156
  (63166, 2151)	0.18546953926341334
  (63166, 8666)	0.3542172419803746
  (63166, 7531)	0.07663745938129565
  (63166, 4487)	0.10061470110625537
  (63166, 6284)	0.09989303

In [92]:
#TFIDF

#### Train and Test Split for Model Building

In [199]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(padded_seq, y, 
                                                    test_size=0.3, 
                                                    random_state=101)

In [200]:
import keras

In [201]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout, Conv1D, Flatten,GlobalAveragePooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [202]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [203]:
from sklearn import datasets
from sklearn.metrics import accuracy_score, f1_score

In [204]:
model = Sequential()
model.add(Embedding(vocab_size +1, 128, input_length = max_length))
model.add(Dropout(0.15))
model.add(Conv1D(300, 5, padding = 'valid', activation = "relu", strides = 1))
model.add(GlobalMaxPool1D())
model.add(Dense(100, activation = "sigmoid"))
model.compile(loss = 'binary_crossentropy ', optimizer = 'adam', metrics = ['accuracy'])
#model.add(Activation('sigmoid'))

In [205]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 125, 128)          8884992   
                                                                 
 dropout_5 (Dropout)         (None, 125, 128)          0         
                                                                 
 conv1d_5 (Conv1D)           (None, 121, 300)          192300    
                                                                 
 global_max_pooling1d_3 (Glo  (None, 300)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
Total params: 9,107,392
Trainable params: 9,107,392
Non-trainable params: 0
____________________________________________

In [206]:
callbacks = [
             EarlyStopping(patience=3),
             ModelCheckpoint(filepath='model-conv1d_v1.h5', save_best_only=True)
            ]

In [207]:
x_train.shape,y_train.shape

((44216, 125), (44216, 100))

In [208]:
# train model
history = model.fit(x_train, y_train,
                    epochs=100,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)

Epoch 1/100
277/277 [==============================] - 106s 368ms/step - loss: 0.1040 - accuracy: 0.1100 - val_loss: 0.0660 - val_accuracy: 0.2298
Epoch 2/100
277/277 [==============================] - 91s 329ms/step - loss: 0.0574 - accuracy: 0.3172 - val_loss: 0.0514 - val_accuracy: 0.3719
Epoch 3/100
277/277 [==============================] - 99s 358ms/step - loss: 0.0454 - accuracy: 0.4353 - val_loss: 0.0437 - val_accuracy: 0.4452
Epoch 4/100
277/277 [==============================] - 98s 353ms/step - loss: 0.0383 - accuracy: 0.5053 - val_loss: 0.0406 - val_accuracy: 0.4740
Epoch 5/100
277/277 [==============================] - 82s 296ms/step - loss: 0.0337 - accuracy: 0.5465 - val_loss: 0.0394 - val_accuracy: 0.4833
Epoch 6/100
277/277 [==============================] - 82s 298ms/step - loss: 0.0298 - accuracy: 0.5806 - val_loss: 0.0389 - val_accuracy: 0.4880
Epoch 7/100
277/277 [==============================] - 78s 283ms/step - loss: 0.0263 - accuracy: 0.6139 - val_loss: 0.0392 

In [209]:
preds = model.predict(x_val)
preds.shape
preds

593/593 [==============================] - 15s 25ms/step


array([[6.36235699e-02, 9.84066210e-05, 1.84868637e-04, ...,
        2.75477214e-05, 6.50058675e-04, 2.50081043e-03],
       [2.91070789e-01, 9.12093237e-05, 7.37332357e-06, ...,
        1.02035627e-02, 2.74409689e-02, 3.48909089e-05],
       [1.44421996e-04, 3.28104496e-01, 6.00373896e-04, ...,
        2.93276444e-05, 3.69136683e-06, 3.25575680e-03],
       ...,
       [1.54418359e-03, 3.71306160e-05, 2.57999054e-05, ...,
        1.01699356e-04, 7.29465783e-01, 1.53918591e-05],
       [5.95430902e-04, 3.27085072e-05, 9.22711659e-03, ...,
        8.87731003e-05, 4.66072379e-04, 6.08436239e-04],
       [3.62475344e-04, 1.07452851e-02, 1.36406961e-05, ...,
        6.10871211e-07, 9.05809156e-06, 3.96120129e-04]], dtype=float32)

In [210]:
# set threshold to 0.45
preds_int = (preds >= 0.5).astype(int)
preds_int

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [211]:
from sklearn.metrics import f1_score

# calculate F1 score
f1_score(y_val, preds_int, average="micro")

0.5584099881371425

In [212]:
#print(clf.intercept_)
#print(clf.coef_)

RSS = np.mean((preds_int-y_val)**2)/(np.std(y_val)**2)
Rsq = 1-RSS

print("RSS per sample = {}".format(RSS))
print("R^2 =            {}".format(Rsq))

RSS per sample = .net         1.196152
ajax         0.857859
algorithm    0.773990
android      0.360529
angularjs    0.375106
               ...   
windows      0.984489
winforms     0.979577
wpf          0.510513
xcode        0.822046
xml          0.885110
Length: 100, dtype: float64
R^2 =            .net        -0.196152
ajax         0.142141
algorithm    0.226010
android      0.639471
angularjs    0.624894
               ...   
windows      0.015511
winforms     0.020423
wpf          0.489487
xcode        0.177954
xml          0.114890
Length: 100, dtype: float64


In [213]:
def infer_tags(q):
    q = clean_string_body(q)
    q = q.lower()
    q_seq = tokenizer.texts_to_sequences([q])
    q_pred = model.predict(q_seq)
    q_pred = (q_pred >= 0.3).astype(int)
    
    return multilabel_binarizer.inverse_transform(q_pred)

In [214]:
# give new question
new_q = "Tf-Idf stands for term frequency inverse document frequency. Term Frequency summarizes how often a given word appears within a document and Inverse Document Frequency downscales words that appear a lot across documents. Tfidf scores are the product of these two terms." 
infer_tags(new_q)

1/1 [==============================] - 0s 258ms/step


[('javascript', 'php')]

In [215]:
array=[]
print("enter no. of texts you wanted to predict?")
n=int(input())
for i in range(n):
  print("Enter your Text:"+str(i+1))
  array.append(str(input()))

#array=["Using neural-networks, deep learning and machine learning makes my life easier","python is easier than javascript while doing machine learning"]
dataframe=pd.DataFrame(array, columns=["Text Title"])
dataframe

enter no. of texts you wanted to predict?
1
Enter your Text:1
are there any available implementations of density or conditional density tree learning greetings i am working on joint and conditional density trees for approximating clique potentials in bayesian


,Text Title
0,are there any available implementations of den...


In [216]:
for i in range(n):
    print("Tags: ", dataframe['Text Title'][i], "\nPredicted tags: ", infer_tags(dataframe['Text Title'][i]))

1/1 [==============================] - 0s 38ms/step
Tags:  are there any available implementations of density or conditional density tree learning greetings i am working on joint and conditional density trees for approximating clique potentials in bayesian 
Predicted tags:  [('.net', 'performance', 'r')]
